In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## OCCLUSION DETECTION

In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)



def pathofimage(path):
  from PIL import Image
  # get image
  filepath = path
  img = Image.open(filepath)
  # get width and height
  width = img.width
  height = img.height
  x = width/2
  y = height/2
  return x,y,width,height



def centerpointandcut(x,y,w,h):
  width = 0.8*w
  height = 0.8*h

  x3, y3 = x - width/2, y - height/2
  x4, y4 = x + width/2, y + height/2
  return x3,y3,x4,y4


def iouscore(x1,y1,x2,y2,x3,y3,x4,y4):
  box1 = x1, y1, x2, y2
  box2 = x3, y3, x4, y4
  x_inter1 = max(x1, x3)
  y_inter1 = max(y1, y3)
  x_inter2 = min(x2, x4)
  y_inter2 = min(y2, y4)
  width_inter = abs(x_inter2 - x_inter1)
  height_inter = abs(y_inter2 - y_inter1)
  area_inter = width_inter * height_inter
  width_box1 = abs(x2 - x1)
  height_box1 = abs(y2 - y1)
  width_box2 = abs(x4 - x3)
  height_box2 = abs(y4 - y3)
  area_box1 = width_box1 * height_box1
  area_box2 = width_box2 * height_box2
  area_union = area_box1 + area_box2 - area_inter
  iou = area_inter / area_union
  ioupercent = iou*100
  return ioupercent


def OcculsionDetector(img):
    image = img
    results = model(image)
    results.print()
    results.xyxy[0]
    df = results.pandas().xyxy[0]
    print("-----------------------------------------------------------------------------------------")
    print("THIS IS THE DATA FRAME OF THE DETECTED OBJECTS")
    print(df)
    print("-----------------------------------------------------------------------------------------")
    print("    ")
    for i in range(0,len(df.axes[0])):
      print("-----------------------------------------------------------------------------------------")
      print("BOUNDING BOX NUMBER ",i)
      print("-----------------------------------------------------------------------------------------")
      x1 = df['xmin'].loc[df.index[i]]
      y1 = df['ymin'].loc[df.index[i]]
      x2 = df['xmax'].loc[df.index[i]]
      y2 = df['ymax'].loc[df.index[i]]
      print(x1,y1,x2,y2)
      print("     ")
      x,y,w,h = pathofimage(image)
      print("    ")
      x3,y3,x4,y4 = centerpointandcut(x,y,w,h)
      IOUPERCENT = iouscore(x1,y1,x2,y2,x3,y3,x4,y4)
      print("THE IOU SCORE IS :",IOUPERCENT)

      if df['name'].loc[df.index[i]] == 'person' or df['name'].loc[df.index[i]] == 'airplane' or df['name'].loc[df.index[i]] == 'bird' or df['name'].loc[df.index[i]] == 'cat' or df['name'].loc[df.index[i]] == 'dog' or df['name'].loc[df.index[i]] == 'umbrella' or df['name'].loc[df.index[i]] == 'backpack' or df['name'].loc[df.index[i]] == 'cow' :
         if IOUPERCENT > 0.8:
           print("----------------------------------------------------------------------------------------")
           print("The given image has OCCULSION")
           print("LETS GO TO THE NEXT IMAGE")
           print("----------------------------------------------------------------------------------------")
           print("      ")
           print("      ")
           return 'Occulsy'
           break
      else:
        print("NO OCCULSION FOUND")
        print("Going to the next image")

      return 'non Occulsy'


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-26 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


## NOISE DETECTION

In [ ]:
import numpy as np
import cv2
from skimage.io import imread
from matplotlib import pyplot as plt
from pylab import *

def entropy(im):
    # Compute normalized histogram -> p(g)
    p = np.array([(im==v).sum() for v in range(256)])
    p = p/p.sum()
    # Compute e = -sum(p(g)*log2(p(g)))
    e = -(p[p>0]*np.log2(p[p>0])).sum()
    return e


def NoiseDetector(img):
    im = imread(img)
    plt.axis('off')
    noise = entropy(im)
    print(noise)
    if noise > 1.5:
      print("not noisy")
      return 'not noisy'
    else:
      print("noisy")
      return 'noisy'
    plt.title(entropy(im))
    plt.show()


## TEXT DETECTION

In [ ]:
!pip install easyocr
import pandas as pd
import easyocr
reader = easyocr.Reader(['en'], gpu = True)
def TextDetector(image):
  results = reader.readtext(image)
  df = pd.DataFrame(results, columns=['bbox','text','conf'])
  if df.empty or len(df.index) < 7:
    return 'NON TEXTY'
  else:
    return 'TEXTY'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## BLUR DETECTION

In [ ]:
import cv2

def calculate_focal_measure(img):
    # convert RGB image to Gray scale image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Measure focal measure score (laplacian approach)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm

def BlurDetector(img):
    imagePath = img
    image = cv2.imread(imagePath)
    focal_measure = calculate_focal_measure(image)

    print("focal-measure-score",focal_measure)
    if focal_measure > 20:
       print("Non Blurry")
       return 'Non Blurry'
       color = (255, 0, 0)
    else:
       print("Blurry")
       return 'Blurry'
       color = (0, 0, 255)


## MAIN FUNCTION

In [ ]:
import cv2
import glob
import shutil
import os
for img in glob.glob("/content/drive/MyDrive/Hampi/Hampi/Site-1-Saasivekaalu_Ganesha/Session 1/Noisy_with_occlusion/*.jpg"):
  if OcculsionDetector(img) == 'Occulsy':
    src_dir = img
    dst_dir = "/content/drive/MyDrive/HampiDublicate/Site-1/session-1/Occlusion_Images"
    shutil.copy(src_dir, dst_dir)


image 1/1: 2368x4208 2 persons
Speed: 470.6ms pre-process, 404.0ms inference, 1.7ms NMS per image at shape (1, 3, 384, 640)


-----------------------------------------------------------------------------------------
THIS IS THE DATA FRAME OF THE DETECTED OBJECTS
          xmin         ymin         xmax         ymax  confidence  class  \
0  2966.751709  1855.417236  3249.857910  2362.648193    0.897945      0   
1  3598.815430  1974.543457  3989.300781  2367.848389    0.886567      0   

     name  
0  person  
1  person  
-----------------------------------------------------------------------------------------
    
-----------------------------------------------------------------------------------------
BOUNDING BOX NUMBER  0
-----------------------------------------------------------------------------------------
2966.751708984375 1855.417236328125 3249.85791015625 2362.648193359375
     
    
THE IOU SCORE IS : 1.2118242964880475
----------------------------------------------------------------------------------------
The given image has OCCULSION
LETS GO TO THE NEXT IMAGE
-----------------------------------

image 1/1: 2368x4208 1 person
Speed: 732.4ms pre-process, 469.4ms inference, 1.8ms NMS per image at shape (1, 3, 384, 640)


-----------------------------------------------------------------------------------------
THIS IS THE DATA FRAME OF THE DETECTED OBJECTS
          xmin        ymin        xmax         ymax  confidence  class    name
0  2884.677979  1429.99939  3289.92749  2356.068359    0.914654      0  person
-----------------------------------------------------------------------------------------
    
-----------------------------------------------------------------------------------------
BOUNDING BOX NUMBER  0
-----------------------------------------------------------------------------------------
2884.677978515625 1429.9993896484375 3289.927490234375 2356.068359375
     
    
THE IOU SCORE IS : 4.393043496661632
----------------------------------------------------------------------------------------
The given image has OCCULSION
LETS GO TO THE NEXT IMAGE
----------------------------------------------------------------------------------------
      
      


image 1/1: 2368x4208 2 persons
Speed: 816.8ms pre-process, 289.5ms inference, 1.8ms NMS per image at shape (1, 3, 384, 640)


...

-----------------------------------------------------------------------------------------
THIS IS THE DATA FRAME OF THE DETECTED OBJECTS
          xmin         ymin         xmax         ymax  confidence  class  \
0   810.627625  1427.328003  1252.956543  2350.881104    0.921919      0   
1  1484.494507  1648.941406  1848.747314  2356.203369    0.904203      0   
2  2692.898193   591.953491  3877.648193  2324.806641    0.895616      0   
3  3753.881348   993.509033  4205.210938  2347.121094    0.850275      0   

     name  
0  person  
1  person  
2  person  
3  person  
-----------------------------------------------------------------------------------------
    
-----------------------------------------------------------------------------------------
BOUNDING BOX NUMBER  0
-----------------------------------------------------------------------------------------
810.6276245117188 1427.3280029296875 1252.95654296875 2350.881103515625
     
    
THE IOU SCORE IS : 4.808771745791304
----

image 1/1: 2368x4208 3 persons
Speed: 609.0ms pre-process, 269.7ms inference, 1.3ms NMS per image at shape (1, 3, 384, 640)


-----------------------------------------------------------------------------------------
THIS IS THE DATA FRAME OF THE DETECTED OBJECTS
          xmin         ymin         xmax         ymax  confidence  class  \
0  1241.505493   789.292175  2723.989502  2358.897217    0.934223      0   
1  1131.151978  1428.654785  1362.817627  2353.309082    0.884986      0   
2  3420.982666   744.042908  4208.000000  2359.027100    0.749714      0   

     name  
0  person  
1  person  
2  person  
-----------------------------------------------------------------------------------------
    
-----------------------------------------------------------------------------------------
BOUNDING BOX NUMBER  0
-----------------------------------------------------------------------------------------
1241.5054931640625 789.2921752929688 2723.989501953125 2358.897216796875
     
    
THE IOU SCORE IS : 29.62616077874667
----------------------------------------------------------------------------------------
Th

image 1/1: 2368x4208 3 persons, 1 skateboard
Speed: 539.8ms pre-process, 233.4ms inference, 1.3ms NMS per image at shape (1, 3, 384, 640)


-----------------------------------------------------------------------------------------
THIS IS THE DATA FRAME OF THE DETECTED OBJECTS
          xmin         ymin         xmax         ymax  confidence  class  \
0   623.736633  1170.692139  1128.814331  2354.020752    0.937248      0   
1     0.312774  1385.343384   135.687943  2223.142334    0.812470      0   
2     0.000000  2110.018066   138.817978  2257.605713    0.612521     36   
3  4167.257812   692.235901  4207.843750   861.612366    0.288279      0   

         name  
0      person  
1      person  
2  skateboard  
3      person  
-----------------------------------------------------------------------------------------
    
-----------------------------------------------------------------------------------------
BOUNDING BOX NUMBER  0
-----------------------------------------------------------------------------------------
623.7366333007812 1170.692138671875 1128.8143310546875 2354.020751953125
     
    
THE IOU SCORE IS : 7